In [ ]:
import signal
from pathlib import Path
import wget

parser_version = '1.6.4'
url = f'https://github.com/nemoware/document-parser/releases/download/{parser_version}/document-parser-{parser_version}.jar'
if not Path(f'document-parser-{parser_version}.jar').is_file():
    wget.download(url)

In [ ]:
import glob
import zipfile

with zipfile.ZipFile(f'./{glob.glob("ДД по практикам*.zip")[0]}',
                     'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
parser_version = '1.6.4'
!java -cp "document-parser-$parser_version/classes;document-parser-$parser_version/lib/*" com.nemo.document.parser.App -i "ДД по практикам\Практика поддержки региональных, розничных продаж и клиентского сервиса\СОГЛАШЕНИЕ о замене стороны.docx"


In [ ]:
import platform
import sys
import os
import json
import subprocess
import pandas as pd
import fnmatch
import base64
import requests
import time

index = 1
number_of_docs = 0
parser_version = '1.6.4'
root = "ДД по практикам"

arrOfAllDocs = []
result = []
result_of_fail = []
paragraph_number = 1
s = [
    "java",
    "-jar",
    f"document-parser-{parser_version}.jar",
    "--server.port=8083"
]
headers = {
    'Content-type': 'application/json',
    'Accept': 'application/json; text/plain'
}

for root, dirnames, filenames in os.walk('ДД по практикам'):
    for filename in fnmatch.filter(filenames, '*.docx'):
        arrOfAllDocs.append(os.path.join(root, filename))
    for filename in fnmatch.filter(filenames, '*.doc'):
        arrOfAllDocs.append(os.path.join(root, filename))

print(
    "Запуск document-parser на 8083 порту, если что0то пойдет не так, то руками УБЕЙТЕ java процесс"
)
java_subprocess = subprocess.Popen(s, creationflags=subprocess.CREATE_NEW_PROCESS_GROUP,
                                   stdout=subprocess.PIPE, encoding="utf-8")
time.sleep(2)
i = 1
while True:
    time.sleep(0.1)
    output_log_spring = java_subprocess.stdout.readline()
    sys.stdout.write("\rПроверка соединения #%i" % i)
    sys.stdout.flush()
    i += 1
    if output_log_spring.find(
            "Started DocumentParserService") != -1:
        print("\nГотово")
        java_subprocess.stdout.close()
        break

print("Запустился успешно")
print("Общее количество документов =", len(arrOfAllDocs))

for docs in arrOfAllDocs:
    try:
        file = open(docs, 'rb')
        encoded_string = base64.b64encode(file.read())
        encoded_string = str(encoded_string)[2:-1]
    except Exception as e:
        print(f"\nОшибка в файле {docs}")
        print(f"при конвертации в base64, исключение = {e.msg}")
        print("=" * 200)
        continue

    response = requests.post(
        "http://localhost:8083/document-parser",
        data=json.dumps({
            "base64Content": encoded_string,
            "documentFileType": docs.split(".")[-1].upper()
        }),
        headers=headers
    )
    resArr = []
    try:
        resArr = response.json()['documents']
    except Exception as e:
        print(f"\nОшибка в файле {docs}")
        print(f"Ответ от парсера {response.json()}")
        print(f"Исключение = {e}")
        print("=" * 200)
        continue

    sys.stdout.write(f"\rПроверка документа под номером {index}")
    sys.stdout.flush()
    index += 1

    document = []
    if resArr:
        document = resArr[0]
    else:
        continue

    if document['documentType'] == "CONTRACT":
        flag = False
        for p in document['paragraphs']:
            if any(z.lower() in p['paragraphHeader']['text'].lower() for z in [
                'ПРЕДМЕТ', 'Общие состояние']) and p['paragraphBody']['length'] > 20:
                result.append(
                    {
                        "path": docs,
                        "name": docs.split("\\")[-1],
                        "documentType": document['documentType'],
                        "offset": p['paragraphBody']['offset'],
                        "text": p['paragraphBody']['text'],
                        "length": p['paragraphBody']['length'],
                        "offsetHeader": p['paragraphHeader']['offset'],
                        "textHeader": p['paragraphHeader']['text'],
                        "lengthHeader": p['paragraphHeader']['length']
                    })
                flag = True
                break

        if flag: continue
        result_of_fail.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": document['paragraphs'][0]['paragraphBody']['offset'],
                "text": "".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in document['paragraphs']),
                "offsetHeader": document['paragraphs'][0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs'])
            })
    elif document['documentType'] == "SUPPLEMENTARY_AGREEMENT":
        #document['paragraphs'][0]['paragraphHeader']['text']
        result.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": document['paragraphs'][0]['paragraphBody']['offset'],
                "text": "".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in document['paragraphs']),
                "offsetHeader": document['paragraphs'][0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs'])
            })
    elif document['documentType'] == "AGREEMENT":
        flag = False
        for p in document['paragraphs']:
            if any(f.lower() in p['paragraphHeader']['text'].lower() for f in
                   ['Предмет соглашения', 'Общие состояние']) and p['paragraphBody']['length'] > 20:
                result.append(
                    {
                        "path": docs,
                        "name": docs.split("\\")[-1],
                        "documentType": document['documentType'],
                        "offset": p['paragraphBody']['offset'],
                        "text": p['paragraphBody']['text'],
                        "length": p['paragraphBody']['length'],
                        "offsetHeader": p['paragraphHeader']['offset'],
                        "textHeader": p['paragraphHeader']['text'],
                        "lengthHeader": p['paragraphHeader']['length']
                    })
                flag = True
                break

        if flag: continue

        # arr_of_paragraphs[
        #     paragraph_number if paragraph_number < len(arr_of_paragraphs) else 0][
        #     'paragraphBody']['text'] if
        # arr_of_paragraphs[paragraph_number if paragraph_number < len(
        #     arr_of_paragraphs) else 0]['paragraphBody']['length'] > 20 else "".join(
        #     str(x['paragraphBody']['text']) for x in document['paragraphs'])

        arr_of_paragraphs = document['paragraphs']
        result_of_fail.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": arr_of_paragraphs[0]['paragraphBody']['offset'],
                "text": "".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in arr_of_paragraphs),
                "offsetHeader": arr_of_paragraphs[0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs'])
            })

#Смерть java процессу!
if platform.system() == 'Windows':
    subprocess.run("TASKKILL /F /PID {pid} /T".format(pid=java_subprocess.pid))
elif platform.system() == 'Linux':
    os.kill(java_subprocess.pid, signal.SIGTERM)
else:
    print('Не известная платформа')
# os.killpg(os.getpgid(java_subprocess.pid), signal.SIGTERM)

writer = pd.ExcelWriter("classifier.xlsx", engine='xlsxwriter')

df = pd.DataFrame(result)
df.to_excel(writer, 'good', engine='xlsxwriter')
sheets_good = writer.sheets['good']
sheets_good.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_fail)
df.to_excel(writer, 'bad', engine='xlsxwriter')
sheets_bad = writer.sheets['bad']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

writer.save()
print("\nФайл создан")
